In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

%matplotlib inline

In [ ]:
df_multipleChoice = pd.read_csv("../input/multipleChoiceResponses.csv",  encoding="ISO-8859-1", low_memory=False)
df_freeform = pd.read_csv("../input/freeformResponses.csv", low_memory=False)
df_schema = pd.read_csv("../input/schema.csv", index_col="Column")

In [ ]:
temp = pd.DataFrame(df_freeform.loc[df_freeform['SalaryChangeFreeForm'].notnull(), 'SalaryChangeFreeForm'])
temp.reset_index(drop=True, inplace=True)

def change_number(df):
    for i in range(df.shape[0]):
        word = df.loc[i, 'SalaryChangeFreeForm']
        if bool(re.findall('\d+', word)):
            if 'incre' in word.lower():
                print('Positive: {}'.format(word))
                df.loc[i, 'SalaryChangeFreeForm'] = 'Good!'
            elif 'decre' in word:
                print('Negative: {}'.format(word))
                df.loc[i, 'SalaryChangeFreeForm'] = 'Not Good!'
            elif '%' in word:
                print('Positive: {}'.format(word))
                df.loc[i, 'SalaryChangeFreeForm'] = 'Good!'
            else:
                pass
        else:
            continue
    return df

In [ ]:
sid = SentimentIntensityAnalyzer()

In [ ]:
def vader_polarity(text):
    score = sid.polarity_scores(word)
    if score['pos'] > score['neg']:
        return 1
    elif score['pos'] < score['neg']:
        return 0
    else:
        return -1

In [ ]:
temp = change_number(temp)
words = temp.SalaryChangeFreeForm.values

positive_answer = []
negagive_answer = []
neutral_answer = []
for word in words:
    judge = vader_polarity(word)
    if judge == 1:
        positive_answer.append(word)
    elif judge == 0:
        negagive_answer.append(word)
    else:
        neutral_answer.append(word)

"hard to say, we have a 30-40% inflation rate in Argentina" and "Increased 100% in the past, but decreased again to the same level as three years ago" would be moved into neutral.

In [ ]:
print(len(positive_answer)-2, len(neutral_answer)+2, len(negagive_answer))

Simple summation of each emotion category shows that positive response is a bit larger than negative.

Let's see all answers, because It is not too big.

In [ ]:
for ans in positive_answer:
    print(ans)

I think "As a phd student on a government research grant I earn less than half of what I did while working." could be moved into negative category. (-1)

In [ ]:
for ans in negagive_answer:
    print(ans)

1. "no salary", "I changed disciplines recently and cannot make a fair comparison.", "No HIKE this Year :(", "No", "Don't want to say", "Not charged", "No change" could be categorized into neutral. (-7)
2. So, the final frequencies of each emotion is following: positve(17), negative(6), neutral(78).

 ## Conclusion
 Of course, It is not easy to say the data science is always helpful to increase salary. But, from output of shell(6), if increase happens because of data science, the amount of increase could be pretty large. (The increase range is (70% ~ 500%)
In other words, learning data science would give an opportunity of high increase in salary!